In [3]:
!pip install -U accelerate
!pip install -U transformers
!pip install -U accelerate



In [4]:
!pip install transformers
!pip install fast_ml==3.68
!pip install datasets
!pip install  torch


In [6]:
import numpy as np
import pandas as pd
from fast_ml.model_development import train_valid_test_split
from transformers import Trainer, TrainingArguments, AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch import nn
from torch.nn.functional import softmax
from sklearn.metrics import classification_report
import datasets
from sklearn.preprocessing import LabelEncoder

In [7]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (f'Device Availble: {DEVICE}')

Device Availble: cpu


In [9]:
data = pd.read_csv('digikala-comments.csv', encoding = 'utf-8')
data.body = data.body.apply(str)
data.title = data.title.apply(str)
data.head()

,id,title,body,created_at,rate,recommendation_status,is_buyer,product_id,advantages,disadvantages,likes,dislikes,seller_title,seller_code,true_to_size_rate
0,53672599,پیشنهاد نمیشود,به درد نمیخوره,23 شهریور 1402,1.0,not_recommended,True,252058,NaN,NaN,0,0,دیجی‌کالا,5A52N,NaN
1,9897229,بسته بندی بد,می‌تونست به عنوان یه کالای فرهنگی بهتر بسته بن...,16 تیر 1399,0.0,recommended,True,252058,['تجربه جالبی بود برام '],['بسته بندی جالبی نداشت'],1,0,دیجی‌کالا,5A52N,NaN
2,38074516,برس ریمل,بسته بندیش خوب بود\r\n کاربرد و کیفیتشم خیلی خ...,26 مرداد 1401,0.0,recommended,True,3331597,NaN,NaN,0,0,آرالیا بیوتی,ADM47,NaN
3,18628562,خوبه و خوشرنگ,به نظرم خوبه فقط یکم ظریفه. از رنگش خوشم اومد ...,28 اسفند 1399,0.0,recommended,True,3331329,NaN,NaN,0,0,اینجاست آ,9ZMCZ,NaN
4,53301258,برس رنگ مو,معمولیه اگه واسه خونه رنگ کردن شخصی میخواین او...,12 شهریور 1402,3.0,recommended,True,3255700,NaN,NaN,0,0,گالری آرایشی به سیما,CDWHA,NaN


In [10]:
data.groupby('recommendation_status').count()

,id,title,body,created_at,rate,is_buyer,product_id,advantages,disadvantages,likes,dislikes,seller_title,seller_code,true_to_size_rate
recommendation_status,,,,,,,,,,,,,,
no_idea,594121,594121,594121,594121,594121,594121,594121,47192,48389,594121,594121,594121,594121,11181
not_recommended,504903,504903,504903,504903,504903,504903,504903,36948,71315,504903,504903,504903,504903,9519
recommended,4162839,4162839,4162839,4162839,4162839,4162839,4162839,513344,234436,4162839,4162839,4162839,4162839,61952


In [11]:
data = data.dropna(subset=['recommendation_status'])

# print data information
print('data information')
print(data.info(), '\n')
# print missing values information
print('missing values stats')
print(data.isnull().sum(), '\n')

# print some missing values
print('some missing values')
print(data[data['recommendation_status'].isnull()].iloc[:5], '\n')

data information
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5261863 entries, 0 to 6156288
Data columns (total 15 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     int64  
 1   title                  object 
 2   body                   object 
 3   created_at             object 
 4   rate                   float64
 5   recommendation_status  object 
 6   is_buyer               bool   
 7   product_id             int64  
 8   advantages             object 
 9   disadvantages          object 
 10  likes                  int64  
 11  dislikes               int64  
 12  seller_title           object 
 13  seller_code            object 
 14  true_to_size_rate      object 
dtypes: bool(1), float64(1), int64(4), object(9)
memory usage: 607.2+ MB
None 

missing values stats
id                             0
title                          0
body                           0
created_at                     0
rate                     

In [12]:
missing_values = ['nan', 'N/A', 'NA', 'None', '']

# Replace missing values with spaces
data['title'] = data['title'].replace(missing_values, ' ')
data['body'] = data['body'].replace(missing_values, ' ')


data['comment'] = data['title'] + ' ' + data['body']


In [13]:
data.head()

,id,title,body,created_at,rate,recommendation_status,is_buyer,product_id,advantages,disadvantages,likes,dislikes,seller_title,seller_code,true_to_size_rate,comment
0,53672599,پیشنهاد نمیشود,به درد نمیخوره,23 شهریور 1402,1.0,not_recommended,True,252058,NaN,NaN,0,0,دیجی‌کالا,5A52N,NaN,پیشنهاد نمیشود به درد نمیخوره
1,9897229,بسته بندی بد,می‌تونست به عنوان یه کالای فرهنگی بهتر بسته بن...,16 تیر 1399,0.0,recommended,True,252058,['تجربه جالبی بود برام '],['بسته بندی جالبی نداشت'],1,0,دیجی‌کالا,5A52N,NaN,بسته بندی بد می‌تونست به عنوان یه کالای فرهنگی...
2,38074516,برس ریمل,بسته بندیش خوب بود\r\n کاربرد و کیفیتشم خیلی خ...,26 مرداد 1401,0.0,recommended,True,3331597,NaN,NaN,0,0,آرالیا بیوتی,ADM47,NaN,برس ریمل بسته بندیش خوب بود\r\n کاربرد و کیفیت...
3,18628562,خوبه و خوشرنگ,به نظرم خوبه فقط یکم ظریفه. از رنگش خوشم اومد ...,28 اسفند 1399,0.0,recommended,True,3331329,NaN,NaN,0,0,اینجاست آ,9ZMCZ,NaN,خوبه و خوشرنگ به نظرم خوبه فقط یکم ظریفه. از ر...
4,53301258,برس رنگ مو,معمولیه اگه واسه خونه رنگ کردن شخصی میخواین او...,12 شهریور 1402,3.0,recommended,True,3255700,NaN,NaN,0,0,گالری آرایشی به سیما,CDWHA,NaN,برس رنگ مو معمولیه اگه واسه خونه رنگ کردن شخصی...


In [14]:
pos = data[data['recommendation_status'] == 'recommended']
nat=data[data['recommendation_status'] == 'no_idea']
neg=data[data['recommendation_status'] == 'not_recommended']

data = pd.concat([pos[:450], neg[:450]], axis=0)

In [15]:
le = LabelEncoder()
data['recommendation_status'] = le.fit_transform(data['recommendation_status'])
data.head()

,id,title,body,created_at,rate,recommendation_status,is_buyer,product_id,advantages,disadvantages,likes,dislikes,seller_title,seller_code,true_to_size_rate,comment
1,9897229,بسته بندی بد,می‌تونست به عنوان یه کالای فرهنگی بهتر بسته بن...,16 تیر 1399,0.0,1,True,252058,['تجربه جالبی بود برام '],['بسته بندی جالبی نداشت'],1,0,دیجی‌کالا,5A52N,NaN,بسته بندی بد می‌تونست به عنوان یه کالای فرهنگی...
2,38074516,برس ریمل,بسته بندیش خوب بود\r\n کاربرد و کیفیتشم خیلی خ...,26 مرداد 1401,0.0,1,True,3331597,NaN,NaN,0,0,آرالیا بیوتی,ADM47,NaN,برس ریمل بسته بندیش خوب بود\r\n کاربرد و کیفیت...
3,18628562,خوبه و خوشرنگ,به نظرم خوبه فقط یکم ظریفه. از رنگش خوشم اومد ...,28 اسفند 1399,0.0,1,True,3331329,NaN,NaN,0,0,اینجاست آ,9ZMCZ,NaN,خوبه و خوشرنگ به نظرم خوبه فقط یکم ظریفه. از ر...
4,53301258,برس رنگ مو,معمولیه اگه واسه خونه رنگ کردن شخصی میخواین او...,12 شهریور 1402,3.0,1,True,3255700,NaN,NaN,0,0,گالری آرایشی به سیما,CDWHA,NaN,برس رنگ مو معمولیه اگه واسه خونه رنگ کردن شخصی...
5,53266157,خوبه,قبلا هم استفاده کردم اگه بلد باشین کار کردن با...,11 شهریور 1402,5.0,1,True,3305270,NaN,NaN,0,0,یانگوم,ERKSP,NaN,خوبه قبلا هم استفاده کردم اگه بلد باشین کار کر...


In [16]:
print (le.classes_)


['not_recommended' 'recommended']


In [17]:
data = data[['recommendation_status', 'comment']]

In [18]:
data.head(20)

,recommendation_status,comment
1,1,بسته بندی بد می‌تونست به عنوان یه کالای فرهنگی...
2,1,برس ریمل بسته بندیش خوب بود\r\n کاربرد و کیفیت...
3,1,خوبه و خوشرنگ به نظرم خوبه فقط یکم ظریفه. از ر...
4,1,برس رنگ مو معمولیه اگه واسه خونه رنگ کردن شخصی...
5,1,خوبه قبلا هم استفاده کردم اگه بلد باشین کار کر...
7,1,پد شستشوی براش خیلی به کارتون میاد قیمتشم مناسبه
8,1,لنز ناخون دقیقا مطابق عکس بود
9,1,خوب برا حالت دادن با اسپری یا پودر عالیه
11,1,عالی مطابق توضیحات بود
12,1,لب ھارو نرم میکنہ و قیمت مناسب


In [19]:
(train_texts, train_labels,
 val_texts, val_labels,
 test_texts, test_labels) = train_valid_test_split(data, target = 'recommendation_status', train_size=0.8, valid_size=0.1, test_size=0.1)

In [20]:
train_texts = train_texts['comment'].to_list()
train_labels = train_labels.to_list()
val_texts = val_texts['comment'].to_list()
val_labels = val_labels.to_list()
test_texts = test_texts['comment'].to_list()
test_labels = test_labels.to_list()

In [21]:
class DataLoader(torch.utils.data.Dataset):
    def __init__(self, sentences=None, labels=None):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

        if bool(sentences):
            self.encodings = self.tokenizer(self.sentences,
                                            truncation = True,
                                            padding = True)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        if self.labels == None:
            item['labels'] = None
        else:
            item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.sentences)


    def encode(self, x):
        return self.tokenizer(x, return_tensors = 'pt').to(DEVICE)

In [22]:
train_dataset = DataLoader(train_texts, train_labels)
val_dataset = DataLoader(val_texts, val_labels)
test_dataset = DataLoader(test_texts, test_labels)

In [23]:
print (train_dataset.__getitem__(0))

{'input_ids': tensor([  101,  1296, 25573, 29823, 14157,  1278, 29836, 29824, 29817,  1278,
        25573, 29825, 29817, 22192,  1297, 22192,  1275, 25573, 23673, 29816,
        14157,  1297, 22192,  1283, 24830, 29841,  1271, 29824, 29817, 14157,
         1271, 15915, 15394, 24830,  1297, 22192,  1273, 22192, 24830, 29823,
         1271, 29836, 15394,  1297, 22192,  1282, 25573, 23673, 22192,  1008,
         1282, 23673, 25573, 22192, 24830,  1278, 29836, 29816, 25573, 17149,
        14157, 14157,  1271, 17149, 25573, 24830,  1271, 25573, 17149,  1282,
        29836, 22192,  1277, 17149, 24830, 15394, 22192,  1024,  1007,  1007,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [24]:
f1 = datasets.load_metric('f1')
accuracy = datasets.load_metric('accuracy')
precision = datasets.load_metric('precision')
recall = datasets.load_metric('recall')
def compute_metrics(eval_pred):
    metrics_dict = {}
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    metrics_dict.update(f1.compute(predictions = predictions, references = labels, average = 'macro'))
    metrics_dict.update(accuracy.compute(predictions = predictions, references = labels))
    metrics_dict.update(precision.compute(predictions = predictions, references = labels, average = 'macro'))
    metrics_dict.update(recall.compute(predictions = predictions, references = labels, average = 'macro'))
    return metrics_dict

In [27]:
id2label = {idx:label for idx, label in enumerate(le.classes_)}
label2id = {label:idx for idx, label in enumerate(le.classes_)}
config = AutoConfig.from_pretrained('distilbert-base-uncased',
                                    num_labels = 2,
                                    id2label = id2label,
                                    label2id = label2id)
model = AutoModelForSequenceClassification.from_config(config)

In [28]:
print (config)

DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "not_recommended",
    "1": "recommended"
  },
  "initializer_range": 0.02,
  "label2id": {
    "not_recommended": 0,
    "recommended": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.40.2",
  "vocab_size": 30522
}



In [29]:
print (model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [31]:
training_args = TrainingArguments(
    output_dir='/content/model',
    num_train_epochs=10,#10
    per_device_train_batch_size=32,#32
    per_device_eval_batch_size=32,#32
    warmup_steps=500,#500
    weight_decay=0.05,
    report_to='none',
    evaluation_strategy='steps',
    # logging_dir='/kagge/working/logs',
    logging_steps=50)

In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
    )

In [33]:
trainer.train()

Step,Training Loss,Validation Loss,F1,Accuracy,Precision,Recall
50,0.693900,0.663641,0.590081,0.600000,0.631336,0.613095
100,0.640900,0.600425,0.602824,0.611111,0.640757,0.623512
150,0.548600,0.499865,0.776786,0.777778,0.776786,0.776786
200,0.423800,0.479414,0.788235,0.788889,0.787976,0.788690


TrainOutput(global_step=230, training_loss=0.5534929482833199, metrics={'train_runtime': 3781.2933, 'train_samples_per_second': 1.904, 'train_steps_per_second': 0.061, 'total_flos': 953765270323200.0, 'train_loss': 0.5534929482833199, 'epoch': 10.0})

In [34]:
eval_results = trainer.predict(test_dataset)

In [35]:
print (eval_results.predictions)

[[-0.06405827 -0.23567152]
 [ 1.7235731  -1.6201117 ]
 [-1.3104761   0.7712039 ]
 [ 1.7911948  -1.7147461 ]
 [-1.1683381   0.68383014]
 [ 1.2266773  -1.2597146 ]
 [ 0.02337394 -0.25895143]
 [ 2.6247354  -2.211808  ]
 [-2.0933669   1.3671334 ]
 [-0.39209288  0.04238042]
 [-2.0372367   1.3730727 ]
 [-0.3315512  -0.00572346]
 [-0.19110492 -0.17475799]
 [ 0.96989155 -1.0228105 ]
 [-1.0311391   0.5750571 ]
 [ 0.27015373 -0.5110712 ]
 [-0.7281509   0.28186154]
 [-1.959023    1.2352898 ]
 [-1.3244165   0.81918377]
 [-0.04574044 -0.24528925]
 [-1.5558014   0.984145  ]
 [ 1.2809403  -1.3069288 ]
 [-0.16374028 -0.13869959]
 [ 2.5359     -2.1694617 ]
 [ 1.3459791  -1.3689058 ]
 [-0.06276534 -0.25221127]
 [ 1.291079   -1.2766082 ]
 [ 0.39223933 -0.60086566]
 [ 0.94283104 -1.0187075 ]
 [-0.97274446  0.5576172 ]
 [-0.26244968 -0.0708451 ]
 [-2.209413    1.4296813 ]
 [-1.6559567   1.0770837 ]
 [ 1.6937894  -1.5855527 ]
 [ 1.913372   -1.7305424 ]
 [ 0.81338996 -0.9467443 ]
 [ 1.5195256  -1.4855422 ]
 

In [36]:
print (eval_results.metrics)

{'test_loss': 0.5275629162788391, 'test_f1': 0.7737556561085972, 'test_accuracy': 0.7777777777777778, 'test_precision': 0.7720258835241414, 'test_recall': 0.7786843447220806, 'test_runtime': 8.8601, 'test_samples_per_second': 10.158, 'test_steps_per_second': 0.339}
